In [ ]:
## Import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

## Loading, Sorting, and Cleaning Data

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values('order_date')

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum',
          'is_weekend': 'max',
          'is_month_start': 'max',
          'is_month_end': 'max',
          'day_of_week_num': 'max',
          'month_num': 'max',
          'week_of_year': 'max',
          'day_of_month': 'max',
          'quarter': 'max'
      })
      .reset_index())

daily_df = daily_df.sort_values('order_date')
daily_df = daily_df.reset_index(drop=True)

forecast_results = list()                                         # to save the forecasts

## Loop through each pizza

for pizza in daily_df['pizza_name_id'].unique():
  pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
  pizza_df = pizza_df.sort_values('order_date')

  if len(pizza_df) < 60:
        continue

  pizza_df['lag_7'] = pizza_df['quantity'].shift(7)
  pizza_df['rolling_mean_7'] = pizza_df['quantity'].rolling(7).mean()
  pizza_df = pizza_df.dropna().reset_index(drop=True)

  features = ['is_weekend', 'is_month_start', 'is_month_end',
              'day_of_week_num', 'month_num', 'week_of_year', 'day_of_month',
              'quarter', 'lag_7', 'rolling_mean_7']
  target = 'quantity'

  X = pizza_df[features]
  y = pizza_df[target]

  train_size = int(len(X) * 0.8)

  X_train_raw = X.iloc[:train_size]
  X_test_raw  = X.iloc[train_size:]
  y_train_raw = y.iloc[:train_size]
  y_test_raw  = y.iloc[train_size:]

  scaler_X = MinMaxScaler()
  scaler_y = MinMaxScaler()

  X_train_scaled = scaler_X.fit_transform(X_train_raw)
  X_test_scaled  = scaler_X.transform(X_test_raw)

  y_train_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1))
  y_test_scaled  = scaler_y.transform(y_test_raw.values.reshape(-1, 1))


  def create_sequences(X, y, window):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])   # ALL features
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

  window_size = 14

  X_train, y_train = create_sequences(X_train_scaled, y_train_scaled, window_size)
  X_test, y_test   = create_sequences(
        np.vstack([X_train_scaled[-window_size:], X_test_scaled]),
        np.vstack([y_train_scaled[-window_size:], y_test_scaled]),
        window_size
    )

  model = Sequential([
    Input(shape=(window_size, X_train.shape[2])),  # n_features
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

  model.compile(optimizer='adam', loss='mae')

  ## Early Stopping
  early_stop = EarlyStopping(
  monitor='val_loss',
  patience=5,
  restore_best_weights=True
  )


  model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=0
)

  preds = scaler_y.inverse_transform(model.predict(X_test))
  actual = scaler_y.inverse_transform(y_test)

  # Daily MAPE
  mape = mean_absolute_percentage_error(actual, preds) * 100

  # Summed MAPE
  actual_sum = actual.sum()
  pred_sum = preds.sum()
  mape_s = abs(actual_sum - pred_sum) / actual_sum * 100

  last_window = np.vstack([X_train_scaled[-window_size:], X_test_scaled])[-window_size:]
  last_date = pizza_df["order_date"].iloc[-1]
  last_qty = pizza_df['quantity'].iloc[-window_size:].values

  future_preds = []

  for i in range(7):
      next_scaled = model.predict(last_window.reshape(1, window_size, -1), verbose = 0)[0, 0]
      next_qty = scaler_y.inverse_transform([[next_scaled]])[0, 0]
      future_preds.append(next_qty)

      # Update window
      next_date = last_date + pd.Timedelta(days=1)


      lag_7 = last_qty[-7]
      rolling_mean_7 = last_qty[-7:].mean()

      new_row = pd.DataFrame([[
            int(next_date.weekday() >= 5),
            int(next_date.is_month_start),
            int(next_date.is_month_end),
            next_date.weekday(),
            next_date.month,
            int(next_date.isocalendar().week),
            next_date.day,
            next_date.quarter,
            lag_7,
            rolling_mean_7
        ]], columns=features)

      new_row_scaled = scaler_X.transform(new_row)
      last_window = np.vstack([last_window[1:], new_row_scaled])
      last_qty = np.append(last_qty[1:], next_qty)
      last_date = next_date

  forecast_sum = int(round(sum(future_preds)))

  forecast_results.append({
        "pizza_name_id": pizza,
        "forecast_quantity_7_days": forecast_sum,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

forecast_df

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73m

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1

,pizza_name_id,forecast_quantity_7_days,MAPE_s,MAPE
0,bbq_ckn_l,19,6.15,59.33
1,calabrese_m,13,3.59,53.88
2,thai_ckn_m,11,26.09,34.31
3,mediterraneo_m,10,10.34,38.77
4,prsc_argla_m,10,7.65,45.15
...,...,...,...,...
85,veggie_veg_m,13,22.27,42.33
86,soppressata_s,9,26.90,18.01
87,mediterraneo_s,11,28.54,30.03
88,ital_veggie_l,8,18.45,16.40


Average MAPE summed: 14.03%  
Average MAPE daily: 40.44%

In [ ]:
ing_df = pd.read_csv('Prepped_Ingredients.csv')
ing_df.columns

Index(['pizza_name_id', 'pizza_name', 'pizza_ingredients',
       'Items_Qty_In_Grams'],
      dtype='object')

In [ ]:
calculation_df = ing_df.merge(
    forecast_df,
    on="pizza_name_id",
    how="left"
)

calculation_df.head()

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams,forecast_quantity_7_days,MAPE_s,MAPE
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0,19.0,6.15,59.33
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0,19.0,6.15,59.33
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0,19.0,6.15,59.33
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0,19.0,6.15,59.33
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0,19.0,6.15,59.33


In [ ]:
calculation_df["required_qty_grams"] = (
    calculation_df["Items_Qty_In_Grams"] * calculation_df["forecast_quantity_7_days"]
)

calculation_df.head()

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams,forecast_quantity_7_days,MAPE_s,MAPE,required_qty_grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0,19.0,6.15,59.33,760.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0,19.0,6.15,59.33,285.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0,19.0,6.15,59.33,380.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0,19.0,6.15,59.33,570.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0,19.0,6.15,59.33,1140.0


In [ ]:
pd.set_option("display.max_rows", None)


In [ ]:
final_ingredients = (
    calculation_df
    .groupby("pizza_ingredients", as_index=False)
    .agg({"required_qty_grams": "sum"})
)

final_ingredients = final_ingredients.sort_values("required_qty_grams", ascending=False)

In [ ]:
final_ingredients.to_csv('Required_Ingredients.csv', index=False)

In [ ]:
ing = pd.read_csv('Required_Ingredients.csv')
ing

,pizza_ingredients,required_qty_grams
0,Chicken,19600.0
1,Red Onions,19360.0
2,Capocollo,15300.0
3,Tomatoes,13330.0
4,Pepperoni,9200.0
5,Bacon,8820.0
6,Mushrooms,8720.0
7,Spinach,7230.0
8,Garlic,6680.0
9,Corn,4690.0
